In [36]:
import random
import numpy as np
import pandas as pd
from collections import Counter

# Data

In [3]:
df = pd.read_excel('GA_task.xlsx', skiprows=1, header=None)
df

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,R,T,R,T,R,T,R,T,R,T,...,R,T,R,T,R,T,R,T,R,T
1,9,22,5,17,5,24,8,40,9,14,...,7,21,9,24,6,37,9,26,10,36
2,3,49,10,13,4,28,5,47,7,45,...,3,10,10,40,7,22,1,39,2,46
3,1,47,7,23,1,18,5,47,6,30,...,8,22,9,24,3,39,6,31,4,16
4,9,30,4,29,5,32,7,18,2,20,...,7,27,7,22,2,49,10,13,5,18
5,5,21,6,27,4,17,7,27,10,42,...,7,50,4,26,4,19,1,20,2,21
6,3,19,8,33,10,31,2,43,1,29,...,10,29,8,17,3,38,10,21,2,33
7,2,18,2,49,8,50,7,33,4,21,...,10,33,7,12,6,24,5,25,8,46
8,7,45,9,39,7,34,1,10,9,35,...,10,11,6,42,8,20,4,18,3,40
9,6,10,1,15,5,38,1,35,7,43,...,5,13,2,24,4,48,6,40,7,27


In [4]:
df.isna().sum().sum()

0

In [5]:
data = df.iloc[1: ]
data

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
1,9,22,5,17,5,24,8,40,9,14,...,7,21,9,24,6,37,9,26,10,36
2,3,49,10,13,4,28,5,47,7,45,...,3,10,10,40,7,22,1,39,2,46
3,1,47,7,23,1,18,5,47,6,30,...,8,22,9,24,3,39,6,31,4,16
4,9,30,4,29,5,32,7,18,2,20,...,7,27,7,22,2,49,10,13,5,18
5,5,21,6,27,4,17,7,27,10,42,...,7,50,4,26,4,19,1,20,2,21
6,3,19,8,33,10,31,2,43,1,29,...,10,29,8,17,3,38,10,21,2,33
7,2,18,2,49,8,50,7,33,4,21,...,10,33,7,12,6,24,5,25,8,46
8,7,45,9,39,7,34,1,10,9,35,...,10,11,6,42,8,20,4,18,3,40
9,6,10,1,15,5,38,1,35,7,43,...,5,13,2,24,4,48,6,40,7,27
10,8,39,9,12,1,29,7,15,1,12,...,6,28,2,39,6,20,3,40,7,43


In [6]:
n_rows, n_cols = data.shape
n_jobs = n_cols // 2
n_rows, n_cols, n_jobs

(11, 100, 50)

In [7]:
jobs = []
for job_id in range(n_jobs):
    r_col = job_id * 2
    t_col = r_col + 1
    job_operations = []
    for operation_id in range(n_rows):
        R = data.iloc[operation_id, r_col]
        T = data.iloc[operation_id, t_col]
        job_operations.append({'resource': int(R), 'time': int(T)})
    jobs.append(job_operations)

In [8]:
len(jobs)

50

In [9]:
jobs[0]

[{'resource': 9, 'time': 22},
 {'resource': 3, 'time': 49},
 {'resource': 1, 'time': 47},
 {'resource': 9, 'time': 30},
 {'resource': 5, 'time': 21},
 {'resource': 3, 'time': 19},
 {'resource': 2, 'time': 18},
 {'resource': 7, 'time': 45},
 {'resource': 6, 'time': 10},
 {'resource': 8, 'time': 39},
 {'resource': 5, 'time': 46}]

In [10]:
POP_SIZE = 30           # Number of different solutions in the generation
GENERATIONS = 100       # Number of epochs
CROSSOVER_RATE = 0.9
MUTATION_RATE = 0.2
TOURNAMENT_SIZE = 3     # Number of best solutions to select from
NUM_SWAPS = 5

In [11]:
num_jobs = len(jobs)
num_ops_per_job = len(jobs[0])
total_ops = num_jobs * num_ops_per_job

In [38]:
def random_chromosome(num_jobs, num_ops_per_job):
    chrom = []
    for job_id in range(num_jobs):
        chrom.extend([job_id] * num_ops_per_job)
    random.shuffle(chrom)
    return chrom

def decode_and_evaluate(chromosome, jobs):
    num_jobs = len(jobs)
    num_ops_per_job = len(jobs[0])
    job_next_op = [0] * num_jobs # tracks the next operation to schedule for each job
    resource_available = {} # dictionary that holds the time when each resource becomes available
    job_end_time = [0] * num_jobs

    # each resource starts available at time 0
    for job_id in range(num_jobs):
        for job in jobs[job_id]:
            resource_available[job['resource']] = 0

    for job_id in chromosome:
        op_idx = job_next_op[job_id]
        if op_idx >= num_ops_per_job:
            continue  # skip if all operations for this job are done
        op = jobs[job_id][op_idx]
        res = op['resource']
        time_for_op = op['time']

        # The operation can start when both the job and the resource are available
        start = max(job_end_time[job_id], resource_available[res]) # holds the earliest possible start time
        finish = start + time_for_op

        job_end_time[job_id] = finish
        resource_available[res] = finish
        job_next_op[job_id] += 1 # move to the next operation for this job

    return max(job_end_time)

def tournament(pop, fitnesses, k):
    selected = random.sample(list(zip(pop, fitnesses)), k) # select k random solutions with their fitness
    selected.sort(key=lambda x: x[1]) # sort by fitness (lowest time)
    return selected[0][0] # best chromosome

def crossover(parent1, parent2):
    size = len(parent1)
    a, b = sorted(random.sample(range(size), 2))
    child = [None]*size
    child[a:b] = parent1[a:b]
    
    current_counts = Counter(child[a:b])
    
    idx = 0
    for i in range(size):
        if child[i] is None:
            while current_counts[parent2[idx]] >= num_ops_per_job:
                idx += 1
            child[i] = parent2[idx]
            current_counts[parent2[idx]] += 1
            idx += 1
    return child

def mutate(chrom, num_swaps):
     # swap two random genes
     for _ in range(num_swaps):
        a, b = random.sample(range(len(chrom)), 2)
        chrom[a], chrom[b] = chrom[b], chrom[a]


In [39]:
population = [random_chromosome(num_jobs, num_ops_per_job) for _ in range(POP_SIZE)]

for gen in range(GENERATIONS):
    fitnesses = [decode_and_evaluate(chrom, jobs) for chrom in population]
    new_population = []
    for _ in range(POP_SIZE):
        
        parent1 = tournament(population, fitnesses, TOURNAMENT_SIZE)
        parent2 = tournament(population, fitnesses, TOURNAMENT_SIZE)
        while parent2 == parent1:
            parent2 = tournament(population, fitnesses, TOURNAMENT_SIZE)
            
        if random.random() < CROSSOVER_RATE:
            child = crossover(parent1, parent2)
        else:
            child = parent1[:]
            
        if random.random() < MUTATION_RATE:
            mutate(child, NUM_SWAPS)
            
        new_population.append(child)
        
    population = new_population

In [40]:
fitnesses = [decode_and_evaluate(chrom, jobs) for chrom in population]
best_idx = np.argmin(fitnesses)
best_chrom = population[best_idx]
best_time = fitnesses[best_idx]
print("Best time:", best_time)
print("Best chromosome:", best_chrom)

Best time: 2432
Best chromosome: [19, 6, 39, 2, 45, 46, 43, 48, 33, 49, 2, 10, 47, 3, 18, 4, 37, 6, 30, 40, 47, 20, 35, 19, 41, 41, 10, 23, 22, 36, 16, 13, 41, 44, 45, 0, 15, 20, 19, 31, 44, 16, 14, 35, 46, 44, 26, 28, 28, 11, 34, 48, 6, 34, 4, 7, 8, 32, 46, 21, 2, 2, 10, 38, 30, 49, 4, 26, 24, 24, 34, 42, 14, 28, 23, 38, 41, 17, 44, 9, 23, 41, 6, 1, 10, 24, 21, 12, 1, 46, 17, 5, 5, 10, 42, 37, 16, 20, 13, 18, 17, 34, 43, 1, 37, 41, 3, 24, 26, 0, 40, 3, 2, 13, 27, 38, 23, 1, 49, 42, 21, 26, 45, 45, 45, 9, 42, 27, 45, 9, 37, 7, 31, 47, 47, 3, 17, 30, 14, 26, 26, 34, 41, 3, 2, 2, 3, 28, 31, 18, 6, 35, 37, 40, 11, 9, 39, 31, 29, 19, 12, 14, 34, 8, 33, 7, 36, 24, 28, 31, 28, 14, 5, 34, 34, 43, 6, 37, 41, 3, 24, 26, 0, 40, 3, 2, 13, 27, 38, 23, 1, 1, 42, 11, 21, 26, 45, 45, 9, 42, 27, 45, 9, 37, 40, 31, 47, 47, 3, 17, 30, 14, 26, 26, 38, 41, 3, 2, 28, 21, 18, 6, 35, 37, 7, 11, 16, 8, 32, 46, 21, 2, 2, 10, 38, 30, 49, 4, 26, 24, 24, 34, 42, 14, 28, 23, 38, 41, 9, 39, 31, 20, 29, 33, 49, 29, 

In [43]:
pop_sizes = [20, 40, 60]
generations_list = [100, 200, 300]
mutation_rates = [0.1, 0.3]
num_swaps_list = [2, 5]

results = []

for POP_SIZE in pop_sizes:
    for GENERATIONS in generations_list:
            for MUTATION_RATE in mutation_rates:
                for NUM_SWAPS in num_swaps_list:

                    population = [random_chromosome(num_jobs, num_ops_per_job) for _ in range(POP_SIZE)]
                    best_time = float('inf')
                    best_chrom = None

                    for gen in range(GENERATIONS):
                        fitnesses = [decode_and_evaluate(chrom, jobs) for chrom in population]
                        new_population = []
                        for _ in range(POP_SIZE):
                            parent1 = tournament(population, fitnesses, TOURNAMENT_SIZE)
                            parent2 = tournament(population, fitnesses, TOURNAMENT_SIZE)
                            while parent2 == parent1:
                                parent2 = tournament(population, fitnesses, TOURNAMENT_SIZE)
                            if random.random() < CROSSOVER_RATE:
                                child = crossover(parent1, parent2)
                            else:
                                child = parent1[:]
                            if random.random() < MUTATION_RATE:
                                mutate(child, NUM_SWAPS)
                            new_population.append(child)
                        population = new_population

                    fitnesses = [decode_and_evaluate(chrom, jobs) for chrom in population]
                    idx = np.argmin(fitnesses)
                    best_time = fitnesses[idx]
                    best_chrom = population[idx]

                    results.append({
                        'POP_SIZE': POP_SIZE,
                        'GENERATIONS': GENERATIONS,
                        'CROSSOVER_RATE': CROSSOVER_RATE,
                        'MUTATION_RATE': MUTATION_RATE,
                        'NUM_SWAPS': NUM_SWAPS,
                        'BEST_TIME': best_time,
                        'BEST_CHROM': best_chrom
                    })

df_results = pd.DataFrame(results)

In [44]:
df_results.sort_values(by='BEST_TIME', ascending=True)

,POP_SIZE,GENERATIONS,CROSSOVER_RATE,MUTATION_RATE,NUM_SWAPS,BEST_TIME,BEST_CHROM
21,40,300,0.9,0.1,5,2253,"[44, 28, 31, 40, 14, 48, 6, 31, 15, 23, 2, 10,..."
19,40,200,0.9,0.3,5,2373,"[18, 42, 46, 49, 22, 46, 39, 45, 8, 34, 32, 49..."
17,40,200,0.9,0.1,5,2400,"[18, 4, 39, 5, 29, 49, 17, 2, 22, 34, 26, 12, ..."
32,60,300,0.9,0.1,2,2417,"[12, 28, 48, 7, 12, 0, 49, 25, 48, 44, 18, 32,..."
35,60,300,0.9,0.3,5,2421,"[41, 22, 0, 2, 45, 9, 38, 33, 35, 49, 49, 36, ..."
27,60,100,0.9,0.3,5,2422,"[33, 26, 4, 9, 43, 32, 33, 26, 4, 9, 33, 11, 2..."
31,60,200,0.9,0.3,5,2424,"[16, 12, 12, 17, 1, 19, 41, 44, 46, 44, 39, 27..."
28,60,200,0.9,0.1,2,2436,"[33, 20, 15, 42, 6, 27, 46, 30, 18, 26, 23, 47..."
0,20,100,0.9,0.1,2,2453,"[25, 44, 28, 33, 45, 21, 48, 25, 40, 46, 6, 28..."
12,40,100,0.9,0.1,2,2455,"[41, 0, 22, 19, 40, 43, 41, 24, 29, 25, 2, 30,..."
